<a href="https://colab.research.google.com/github/saludwig26/Data-science-Team-5/blob/master/azubi_evaluation_uni_mannheim/code/Projekt_Notebook_Sabrina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive
%cd ML TensorFlow
%cd azubi_evaluation_uni_mannheim
%cd data

Mounted at /content/drive/
/content/drive/MyDrive
/content/drive/MyDrive/ML TensorFlow
/content/drive/MyDrive/ML TensorFlow/azubi_evaluation_uni_mannheim
/content/drive/MyDrive/ML TensorFlow/azubi_evaluation_uni_mannheim/data


In [ ]:
#import csv
#with open('data_MailsAndPC_cleaned.csv', 'r') as file:
#    reader = csv.reader(file)
#    for row in reader:
#        print(row)

In [ ]:
#import io
#azubi_df = pd.read_csv('data_MailsAndPC_cleaned.csv')
#print(azubi_df)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import pandas as pd

azubi_df = pd.read_csv('data_MailsAndPC.csv')
print(azubi_df)

     Unnamed: 0      id  ... a4s3i4a a4s3i4b
0            60  111015  ...     1.0    19.0
1            61  111023  ...     5.0    21.0
2            62  111036  ...     2.0    40.0
3            63  111049  ...     7.0    29.0
4            64  111051  ...     0.0     5.0
..          ...     ...  ...     ...     ...
775         863  631294  ...     0.0    51.0
776         864  631306  ...     1.0    45.0
777         865  631319  ...     3.0    45.0
778         866  631321  ...     3.0    20.0
779         867  631334  ...     0.0    33.0

[780 rows x 43 columns]


In [ ]:
azubi_df

In [13]:
#Christopher's cleaning to 548
def dfreplace(df, *args, **kwargs):
    s = pd.Series(df.values.flatten())
    s = s.str.replace(*args, **kwargs)
    return pd.DataFrame(s.values.reshape(df.shape), df.index, df.columns)

def cleanupDataframe(azubi_df):
    #Remove unneccesary columns
    azubi_df = azubi_df.drop(columns=['Unnamed: 0', 'id'])
    #Remove "cr". Assumption made, that there are no real words containing cr in the mails.
    azubi_df["S1_mail"] = azubi_df["S1_mail"].str.replace("cr", "")
    azubi_df["S2_mail"] = azubi_df["S2_mail"].str.replace("cr", "")
    azubi_df["S3_mail"] = azubi_df["S3_mail"].str.replace("cr", "")
    #Drop NA rows to only have full datasets
    azubi_df = azubi_df.dropna()
    #Select all rows without '99' | ~ inverts the operator
    dropThisNumber = 99
    azubi_df = azubi_df[~azubi_df.eq(dropThisNumber).any(1)]
    #Reset the index of the Dataframe
    azubi_df = azubi_df.reset_index(drop=True)
    return azubi_df

def makeList(azubi_df):
  azubi_df_list = azubi_df.values.tolist()

  #Sentence S1
  s1MailSentences = azubi_df["S1_mail"].tolist()
  #Labels S1
  s1MailA3Labels = azubi_df["a3s1"].tolist()
  s1MailA4Labels = azubi_df["a4s1"].tolist()
  #Sentence S2
  s2MailSentences = azubi_df["S2_mail"].tolist()
  #Labels S2
  s2MailA3Labels = azubi_df["a3s2"].tolist()
  s2MailA4Labels = azubi_df["a4s2"].tolist()
  #Sentence S3
  s3MailSentences = azubi_df["S3_mail"].tolist()
  #Labels S3
  s3MailA3Labels = azubi_df["a3s3"].tolist()
  s3MailA4Labels = azubi_df["a4s3"].tolist()

  return azubi_df_list , s1MailSentences, s1MailA3Labels, s1MailA4Labels, s2MailSentences, s2MailA3Labels, s2MailA4Labels, s3MailSentences, s3MailA3Labels, s3MailA4Labels



In [14]:
azubi_df = cleanupDataframe(azubi_df)
azubi_df

azubi_df_list , s1MailSentences, s1MailA3Labels, s1MailA4Labels, s2MailSentences, s2MailA3Labels, s2MailA4Labels, s3MailSentences, s3MailA3Labels, s3MailA4Labels = makeList(azubi_df)


In [ ]:
azubi_df #548 participants #after cleaning

In [16]:
#save cleaned df to csv 'azubi_dfnew.csv'
pd.DataFrame.to_csv(azubi_df)
print(azubi_df)
dfnew = pd.DataFrame(azubi_df)
dfnew.to_csv('azubi_dfnew.csv', header=False, index=False)

                                               S1_mail  ... a4s3i4b
0    Hallo Fr. Meier,leider konnte ich die von Ihne...  ...    19.0
1    Ihre Antwort...Der EP Plan weicht viel zusehr ...  ...    21.0
2    bitte kommen Sie rüber und erklären mir es selbst  ...    40.0
3    Sehr geehrte Frau Meier,leider konnte Ich Ihre...  ...    29.0
4    Hallo Frau Meier,das Controlling hat für das J...  ...    75.0
..                                                 ...  ...     ...
543  Hallo Frau Meier, ich habe die Aufgabe so gut ...  ...    35.0
544  Hallo Frau Meier,ich habe mich an der Aufgabe ...  ...    45.0
545  Ihre Antwort...Hallo Frau Meier,leider habe ic...  ...    45.0
546  Guten Tag Frau Susanne Meier,ich habe bereits ...  ...    20.0
547  Hallo Frau Meier,anbei sende ich Ihnen die Tab...  ...    33.0

[548 rows x 41 columns]


In [17]:
#importing packages 
import tensorflow as tf
import csv
import random
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

#Global Hyperparameters
vocab_size = 5000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [18]:
#stopwords
import nltk 
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))

print(stop_words)

#allerdings Wörter wie "nicht" enthalten

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'viel', 'demselben', 'allen', 'ihrem', 'seinen', 'deines', 'nur', 'wieder', 'des', 'alles', 'im', 'welchen', 'euren', 'dann', 'sie', 'auch', 'dass', 'hat', 'eurem', 'anderen', 'anderer', 'bin', 'dessen', 'hier', 'hinter', 'daß', 'kann', 'meines', 'unter', 'welche', 'dieser', 'jenem', 'zu', 'anderem', 'anders', 'einer', 'vor', 'jenen', 'könnte', 'auf', 'oder', 'hatte', 'gegen', 'keine', 'solchem', 'selbst', 'seine', 'einmal', 'meinen', 'wollte', 'deinem', 'jedem', 'die', 'für', 'derselben', 'jener', 'ihr', 'man', 'ihre', 'werden', 'aber', 'aller', 'seines', 'es', 'dieselben', 'jeden', 'seinem', 'warst', 'jedes', 'über', 'er', 'in', 'meine', 'unsere', 'habe', 'sehr', 'ihnen', 'will', 'eine', 'also', 'dazu', 'einen', 'einigem', 'zwischen', 'was', 'bist', 'diesen', 'hatten', 

In [19]:
#creating lists by putting the sentences and labels into an array #placing the sequences in square brackets 

s1MailSentences = []
s1MailA3Labels = []
s1MailA4Labels = [] 

s2MailSentences = [] 
s2MailA3Labels = [] 
s2MailA4Labels = [] 

s3MailSentences = []
s3MailA3Labels = [] 
s3MailA4Labels = []


In [20]:
with open("azubi_dfnew.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        s1MailSentences = row[0]
        s2MailSentences = row[1] 
        s3MailSentences = row[2] 
        s1MailA3Labels.append(row[3]) #elements in lists
        s2MailA3Labels.append(row[4])
        s3MailA3Labels.append(row[5])
        s1MailA4Labels.append(row[6]) 
        s2MailA4Labels.append(row[7])
        s3MailA4Labels.append(row[8])

        #for word in stopwords:
        #    token = " " + word + " " #word =stopword gets tokenized
        #    sentence = sentence.replace(token, " ") #replace token with space
        #    sentence = sentence.replace("  ", " ") #to avoid the creation of non-existing words?
        #sentences.append(sentence) #adding sentence to thte list sentences which contain at the end 2225 items


#print(s1MailSentences)
print(row[0])
print(row[1])
print(row[2])
print(row[3])
print(row[4])
print(row[5])
print(len(s3MailA3Labels))
print(s3MailA3Labels)
print(len(s3MailSentences)) #Output: 217? and 548 items
print(s3MailSentences[0])

Hallo Frau Meier,anbei sende ich Ihnen die Tabelle zurück.Mir ist es gelungen, die Abweichungen der Soll- und Ist-Werte und auch die Sollwerte der letzten 3 Lieferanten zu berechnen.Abweichungen mit größer/gleich 20 % habe ich farbig markiert.Leider konnte ich keine Gründe für diese Abweichungen finden.Mein Vorschlag wäre allerdings die geplante Absatzmenge aufgrund des realisierten Absatzes anzupassen.Ebenfalls würde ich einen Lieferantenvergleich vorschlagen, um durch günstigere Lieferanten eventuell Kosten einzusparen und somit die Planmengen zu erreichen. GrüßeMelissa Fischer 
Hallo Herr Neumann,anbei erhalten Sie die Tabelle der Nutzwertanalyse zurück.Laut dieser Tabelle sollten wir wolrd birdy bei dem Lieferanten aus der Tschechei beziehen, da dieser Punktemäsig weit vorne liegt und auch in puncto Qualität und Preis-Leistungsverhältnis überzeugt.Leider kann ich Ihnen keine weiteren Infos liefern, da ich mehr Zeit benötigt hätte.GRüßeMelidssa Fischer 
Hallo Herr Neumann,anbei die 

In [ ]:
azubi_df_list , s1MailSentences, s1MailA3Labels, s1MailA4Labels, s2MailSentences, s2MailA3Labels, s2MailA4Labels, s3MailSentences, s3MailA3Labels, s3MailA4Labels = makeList(azubi_df)
azubi_df_list 

#azubi2_df = []
#s1MailSentences, s1MailA3Labels, s1MailA4Labels, s2MailSentences, s2MailA3Labels, s2MailA4Labels, s3MailSentences, s3MailA3Labels, s3MailA4Labels = makeList(azubi2_df)


In [22]:
#merging all Mails together into a list
Sentences = s1MailSentences + s2MailSentences + s3MailSentences 
Sentences
print(len(Sentences))
#Output: 3 tasks x 548 mails= 1644 mails

1644


In [23]:
#merging all Labels together into a list
Labels = s1MailA3Labels + s1MailA4Labels + s2MailA3Labels + s2MailA4Labels + s3MailA3Labels + s3MailA4Labels
Labels
print(len(Labels))

3288


In [24]:
#whole sample 1644
train_size = 1194     #548*3 emails
test_size = 225       #75*3 emails
validation_size = 225 #75*3 emails

print(train_size)
print(test_size)
print(validation_size)

1194
225
225


In [47]:
def trainDevTestSplit(Sentences, train_size):
    random.shuffle(Sentences)
    for x in range(train_size):
        Sentences.append(Sentences[x][0])
        Labels.append(Sentences[x][0]) 

    train_sentences = Sentences[0:1194]
    train_labels = Labels[0:1194]
    test_sentences = Sentences[1194:1419]
    test_labels = Labels[1194:1419]
    validation_sentences = Sentences[1419:1644]
    validation_labels = Labels[1419:1644]
    return train_sentences, train_labels, validation_sentences, validation_labels, test_sentences, test_labels

#print(train_size)
#print(len(train_sentences))
#print(train_sentences)
#print(len(train_labels))

#print(len(validation_sentences))
#print(len(validation_labels))

#print(len(test_sentences))
#print(len(test_labels))



In [49]:
train_sentences, train_labels, validation_sentences, validation_labels, test_sentences, test_labels = trainDevTestSplit(Sentences, train_size)

In [ ]:
train_sentences #hier sieht man die einzelnen Buchstaben wie 'H'
#print(len(train_sentences))

In [52]:
#tokenize data
def tokenizeData(vocab_size, oov_tok, train_sentences, max_length, padding_type, trunc_type):
    tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
    tokenizer.fit_on_texts(train_sentences)
   
    word_index = tokenizer.word_index
    vocab_size=len(word_index)
    
    sequences = tokenizer.texts_to_sequences(train_sentences)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    return padded

#print(len(word_index))
#print(word_index)

In [32]:
padded = tokenizeData(vocab_size, oov_tok, train_sentences, max_length, padding_type, trunc_type)

In [33]:
padded

array([[ 13,  16,  17, ...,   0,   0,   0],
       [ 13,  16,  17, ...,   0,   0,   0],
       [  6,   0,   0, ...,   0,   0,   0],
       ...,
       [  6,   0,   0, ...,   0,   0,   0],
       [ 34, 165,  40, ...,   0,   0,   0],
       [ 13,  16,  17, ...,   0,   0,   0]], dtype=int32)

In [ ]:
validation_sentences 

In [40]:
validation_sequences = Tokenizer.texts_to_sequences(validation_sentences) #liegt die Fehlermeldung vielleicht an den einzelnen Buchstaben?
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

print(len(validation_sequences))
print(validation_padded.shape)


TypeError: ignored

In [44]:
test_sequences = Tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

print(len(test_sequences))
print(test_padded.shape)

TypeError: ignored